<a href="https://colab.research.google.com/github/joaopamaral/tcc_puc_cdbd/blob/master/EDA_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip -q install kaggle --upgrade

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cp /gdrive/My\ Drive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [8]:
!kaggle competitions download -c reducing-commercial-aviation-fatalities

100% 1.67G/1.67G [00:32<00:00, 61.2MB/s]
100% 1.67G/1.67G [00:32<00:00, 55.4MB/s]
100% 435M/435M [00:09<00:00, 49.9MB/s]
100% 435M/435M [00:09<00:00, 50.1MB/s]
100% 41.0M/41.2M [00:00<00:00, 40.1MB/s]
100% 41.2M/41.2M [00:00<00:00, 52.9MB/s]


In [0]:
!cp {sample_submission.csv.zip,train.csv.zip,test.csv.zip} /gdrive/My\ Drive/Kaggle/reducing-commercial-aviation-fatalities/

In [0]:
import pandas as pd

In [0]:
train_ds = pd.read_csv('train.csv.zip')

In [13]:
train_ds.set_index(['crew', 'seat', 'experiment']).plot().head()

time   eeg_fp1  ...         gsr  event
crew seat experiment                      ...                   
1    1    CA          0.011719  -5.28545  ...  388.829987      A
          CA          0.015625  -2.42842  ...  388.829987      A
          CA          0.019531  10.67150  ...  388.829987      A
          CA          0.023438  11.45250  ...  388.829987      A
          CA          0.027344   7.28321  ...  388.829987      A

[5 rows x 25 columns]